In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20202201
paper_name = 'batova_schuller_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [53]:
original_data = pd.read_excel('raw_data/1471-2164-11-153-S1.xlsx', sheet_name='Sheet1', header=None)

In [54]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 507 x 1


In [55]:
original_data.head()

,0
0,YDR532C
1,sl
2,‐
3,YLR244C
4,‐


In [57]:
t = pd.DataFrame(data = {'orf': original_data.iloc[0::3,0].values, 
                                     'data1': original_data.iloc[1::3,0].values, 
                                     'data2': original_data.iloc[2::3,0].values})

In [58]:
original_data = t.copy()

In [59]:
original_data['orf'] = original_data['orf'].astype(str)

In [60]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [61]:
typo_fixes = {'YJL206CA':'YJL206C-A','YFR031CA':'YFR031C-A','YER087CA':'YER087C-A',
              'YMR031WA':'YMR031W-A','YML081CA':'YML081C-A','YER014CA':'YER014C-A'}

In [62]:
original_data['orf'] = original_data['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)

In [63]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [64]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf, data1, data2]
Index: []


In [65]:
data_switch = {'+': 0, 'sl': -1, '‐': -2}
for c in ['data1','data2']:
    original_data[c] = original_data[c].apply(lambda x: data_switch[x])

In [66]:
original_data.head()

,orf,data1,data2
index_input,,,
0,YDR532C,-1,-2
1,YLR244C,-2,-2
2,YMR060C,-1,-2
3,YOR147W,-2,-2
4,YOL027C,-1,-2


In [67]:
original_data.set_index('orf', inplace=True)

In [68]:
original_data = original_data.groupby(original_data.index).mean()

In [69]:
original_data.shape

(169, 2)

# Prepare the final dataset

In [70]:
data = original_data.copy()

In [71]:
dataset_ids = [153,437]
datasets = datasets.reindex(index=dataset_ids)

In [72]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [73]:
data.head()

dataset_id,153,437
data_type,value,value
orf,,
YAL012W,-2,-2
YBL025W,-1,-2
YBL093C,0,-2
YBL099W,-1,-1
YBL100C,-1,-1


## Subset to the genes currently in SGD

In [74]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [75]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,153,437
,data_type,value,value
gene_id,orf,,
10,YAL012W,-2,-2
113,YBL025W,-1,-2
181,YBL093C,0,-2
187,YBL099W,-1,-1
188,YBL100C,-1,-1


# Normalize

In [79]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [80]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [81]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,153,437,153,437
,data_type,value,value,valuez,valuez
gene_id,orf,,,,
10,YAL012W,-2,-2,-8.989670,-5.640817
113,YBL025W,-1,-2,-4.494835,-5.640817
181,YBL093C,0,-2,0.000000,-5.640817
187,YBL099W,-1,-1,-4.494835,-2.820408
188,YBL100C,-1,-1,-4.494835,-2.820408


# Print out

In [82]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [83]:
from IO.save_data_to_db3 import *

In [84]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/2 [00:00<?, ?it/s]

Deleting all datasets for PMID 20202201...
Inserting the new data...


100%|██████████| 2/2 [00:00<00:00,  2.42it/s]

Updating the data_modified_on field...
